In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-01,France,131041.0,0.0,24777.0,0.0,0.0,0.0
0,2020-05-01,China,84051.0,0.0,4667.0,0.0,0.0,0.0
0,2020-05-01,Italy,207500.0,0.0,28274.0,0.0,0.0,0.0
0,2020-05-01,Spain,214814.0,0.0,24966.0,0.0,0.0,0.0
0,2020-05-01,United States,1098518.0,0.0,65542.0,0.0,0.0,0.0
0,2020-05-01,World,3295073.0,0.0,239208.0,0.0,0.0,0.0
0,2020-05-01,United Kingdom,175740.0,0.0,27528.0,0.0,0.0,0.0
0,2020-05-01,Germany,159970.0,0.0,6368.0,0.0,0.0,0.0
0,2020-05-01,Iran,95676.0,0.0,6107.0,0.0,0.0,0.0
0,2020-05-01,Turkey,122827.0,0.0,3255.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-02,France,131041.0,130185.0,24777.0,24594.0,856.0,183.0
0,2020-05-02,China,84051.0,83959.0,4667.0,4637.0,92.0,30.0
0,2020-05-02,Italy,207500.0,207428.0,28274.0,28236.0,72.0,38.0
0,2020-05-02,Spain,214814.0,215216.0,24966.0,24824.0,-402.0,142.0
0,2020-05-02,United States,1098518.0,1103781.0,65542.0,65068.0,-5263.0,474.0
0,2020-05-02,World,3295073.0,3307600.0,239208.0,238431.0,-12527.0,777.0
0,2020-05-02,United Kingdom,175740.0,177454.0,27528.0,27510.0,-1714.0,18.0
0,2020-05-02,Germany,159970.0,161703.0,6368.0,6575.0,-1733.0,-207.0
0,2020-05-02,Iran,95676.0,95646.0,6107.0,6091.0,30.0,16.0
0,2020-05-02,Turkey,122827.0,122392.0,3255.0,3258.0,435.0,-3.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")